# Temperature Anomalies Calculation Notebook.


In [377]:
############## Script to calculate radiation and temperature anomalies for visualization #############################
#
# This script will output a new netCDF file to visualize the following new variables in Visit:
#
# TOA Net Flux Anomalies. Uses all months' data from 2000-2015. These variables are names in the output file.
# toa_glob = TOA differences from the global average from 2000-2015
# toa_loc = TOA differences from each lat,lon averages from 2000-2015
# toa_m_glob = TOA differences from the global average by month from 2000-2015
# toa_m_loc = TOA differences from each lat,lon averages by month from 2000-2015
# toa_base = TOA differences from the temperature in year 2000 at each lat/lon
#
# Skin Temperature Anomalies. Uses Aux skin temp data from 2000-2015. These variables are names in the output file.
# skin_glob = Skin Temp differences from the global average from 2000-2015
# skin_loc = Skin Temp differences from each lat,lon averages from 2000-2015
# skin_m_glob = Skin Temp differences from the global average by month from 2000-2015
# skin_m_loc = Skin Temp differences from each lat,lon averages by month from 2000-2015
# skin_base = Skin Temp differences from the temperature in year 2000 at each lat/lon
# 
######################################################################################################################


########### PLEASE add notes on ANY EDITS you make here on top of others' so we can keep track of everything!! #######
#
# # # Notes from Kristen 4/13/16 # # #
#
# This script can be used as the basis for calculating climate related anomalies from other netCDF files we download.
# It can also be used as part of a new python script to generate the visit output images as we have done in class the 
# last few weeks. Written in Python 2.7 by the way.
#
# If opened as a new netCDF file, it for some reason does not create 2D plots, though the data is there
# and shows in the legends. If someone can fix this that would be great. Also the script itself isn't the neatest so 
# if anyone has time and is willing to clean it up, that could be nice too. Just simplify some of the code if there
# are better functions I did not know about and wrote something stupid long for. 
#
# Let's add the visit animation capabilities from this data directly in the script so it doesn't have to be exported 
# as a new netCDF file. I wrote this in the iPython Notebook so that I could check a bunch of stuff along the way. When
# modifying it as a .py file just export from the notebook as .py and remove the items that are just extra. Please do
# keep all the comments and notes here and add on to existing notes as you update and upload the file to Dropbox. 
#
#
######################################################################################################################

### Reading in the data

Here I am just getting libraries & reading in the data. Also looking at it. Need to add in section for importing Visit so that we can generate the plates for our movie straight from the code rather than a new netCDF file. Printing the dataset allowed me to see the different variables available. Which I pulled out for later use.

In [3]:
#get things
import numpy as np
import pandas as pd 
from netCDF4 import Dataset 
import numpy.ma as ma 

filename = "/Users/kristenbrown/Desktop/SciViz/project/CERES_SYN1.nc"
ds = Dataset(filename, mode="r")

In [379]:
print(ds) # checking the data set variables, dimensions, sizes, etc so I can use that info later

<type 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    title: CERES SYN1deg Products - Monthly Means
    institution: NASA/LaRC (Langley Research Center) Hampton, Va
    Conventions: CF-1.4
    comment: Data is from East to West and South to North. See values in latitude and longitudes dimensions.
    version: This is version 3A: May 2, 2014
    Fill_Value: Fill Value is -999.0
    dimensions(sizes): lon(360), lat(180), time(182)
    variables(dimensions): float32 lon(lon), float32 lat(lat), int32 time(time), float32 toa_net_all_mon(time,lat,lon), float32 toa_net_clr_mon(time,lat,lon), float32 aux_skint_mon(time,lat,lon)
    groups: 



In [6]:
#pull out relevant data
time = ds.variables['time'][:]
lat = ds.variables['lat'][:]
lon = ds.variables['lon'][:]
toa = ds.variables['toa_net_all_mon'][:,:,:]
skin = ds.variables['aux_skint_mon'][:,:,:]

# this part I was just checking the attributes for each variable so I could save it all the same again later. 
print(ds.variables['time']) 
print time[0]

<type 'netCDF4._netCDF4.Variable'>
int32 time(time)
    long_name: time
    units: days since 2000-03-01 00:00:00
    delta_t: 0000-00-01 00:00:00
unlimited dimensions: 
current shape = (182,)
filling off

14


## Calculating anomalies

Here is where the calculations begin. 

### Local deviation: _loc variable
The very first section (below) calculates the average TOA or Skin Temp at each latitude/longitude position over the entire timecourse (2000-2015). This is used as our "baseline". Each time point is then "normalized" by this baseline. 


TOA at position (1,1) in month 1/2000 - TOA at position (1,1) averaged over all months 2000-2015.

In [381]:
#calculate the average over time at each lat/lon position
toaAvgLL = np.mean(toa[:,:,:],0)
skinAvgLL = np.mean(skin[:,:,:],0)

#calculate the deviation from average
toa_loc = toa - toaAvgLL
skin_loc = skin - skinAvgLL

#what is the min/max (for color table purposes it's good to know)
print "TOA min deviation: ", ma.min(toa_loc), "\nTOA max deviation: ", ma.max(toa_loc)
print "\nSkin temp min deviation: ", ma.min(skin_loc), "\nSkin temp max deviation: ", ma.max(skin_loc)

TOA min deviation:  -182.563 
TOA max deviation:  261.303

Skin temp min deviation:  -37.198 
Skin temp max deviation:  40.537


### Global deviation: _glob variable

Here is the subtraction of the global average over all time 2000-2015. So this is a single number over the entire globe in 15 years. 

TOA at position (1,1) in month 1/2000 - TOA averaged over whole globe & all months 2000-2015

In [382]:
#calculate global average 
toaAvg = ma.average(toa)
skinAvg = ma.average(skin)
print "Global average TOA: ", toaAvg, "\nGlobal average Skin Temp: ", skinAvg

#calculate difference from global average
toa_glob = toa - toaAvg
skin_glob = skin - skinAvg

#what is the min/max (for color table purposes it's good to know)
print "\nTOA min deviation: ", ma.min(toa_glob), "\nTOA max deviation: ", ma.max(toa_glob)
print "\nSkin temp min deviation: ", ma.min(skin_glob), "\nSkin temp max deviation: ", ma.max(skin_glob)

Global average TOA:  -25.9306362773 
Global average Skin Temp:  278.866281373

TOA min deviation:  -188.763 
TOA max deviation:  222.131

Skin temp min deviation:  -84.5329 
Skin temp max deviation:  40.3689


## Calculating monthly averages & deviations

Here we are taking into account the natural variation between months. The cell below just gathers all the indices corresponding to each month over all available (complete) years.

In [383]:
#pull out which indices would correspond to the same month in our dataset
months=np.array([0,1,2,3,4,5,6,7,8,9,10,11])
all_months=np.array([0,1,2,3,4,5,6,7,8,9,10,11])

for i in range(1,15): #each month occurs 15 times except for two months extra
    months+=12
    all_months=np.vstack((all_months, months))

all_months = np.transpose(all_months) #so that it is divided properly

print(all_months) #check that it works, just missing the last two months which I add back in later

[[  0  12  24  36  48  60  72  84  96 108 120 132 144 156 168]
 [  1  13  25  37  49  61  73  85  97 109 121 133 145 157 169]
 [  2  14  26  38  50  62  74  86  98 110 122 134 146 158 170]
 [  3  15  27  39  51  63  75  87  99 111 123 135 147 159 171]
 [  4  16  28  40  52  64  76  88 100 112 124 136 148 160 172]
 [  5  17  29  41  53  65  77  89 101 113 125 137 149 161 173]
 [  6  18  30  42  54  66  78  90 102 114 126 138 150 162 174]
 [  7  19  31  43  55  67  79  91 103 115 127 139 151 163 175]
 [  8  20  32  44  56  68  80  92 104 116 128 140 152 164 176]
 [  9  21  33  45  57  69  81  93 105 117 129 141 153 165 177]
 [ 10  22  34  46  58  70  82  94 106 118 130 142 154 166 178]
 [ 11  23  35  47  59  71  83  95 107 119 131 143 155 167 179]]


### Monthly deviations: _m_glob and _m_loc variables

This section calculates a similar variable as before, except we average by individual months over all time instead of by all months over all time. 

toa_m_loc

TOA at position (1,1) in month 1/2000 - TOA at position (1,1) averaged over all Januaries from 2000-2015


toa_m_glob

TOA at position (1,1) in month 1/2000 - Global average TOA averaged over all Januaries from 2000-2015

In [384]:
#initialize variables in same shape as toa/skin arrays. 
toa_m_loc = np.empty([182,180,360])
toa_m_glob = np.empty([182,180,360])
skin_m_loc = np.empty([182,180,360])
skin_m_glob = np.empty([182,180,360])

#calculate monthly averages and deviations from those averages
for i in range(1,13):
    #just getting the right months set up
    if i is 1:
        submonths = np.concatenate((all_months[i-1],[180])) #need to add on the extra month
    elif i is 2: 
        submonths = np.concatenate((all_months[i-1],[181])) #need to add on the extra month       
    else:
        submonths = all_months[i-1] #all others were just 15 years exactly
    
    #actual calculation for each month, stored in the same places as original data
    toa_m_loc[submonths,:,:] = toa[submonths,:,:] - np.mean(toa[submonths,:,:],0)
    toa_m_glob[submonths,:,:] = toa[submonths,:,:] - ma.average(toa[submonths,:,:])
    skin_m_loc[submonths,:,:] = skin[submonths,:,:] - np.mean(skin[submonths,:,:],0)
    skin_m_glob[submonths,:,:] = skin[submonths,:,:] - ma.average(skin[submonths,:,:])

#what is the min/max (for color table purposes it's good to know)
print "Deviation from global averages by month...."
print "\nTOA min deviation: ", ma.min(toa_m_glob), "\nTOA max deviation: ", ma.max(toa_m_glob)
print "\nSkin temp min deviation: ", ma.min(skin_m_glob), "\nSkin temp max deviation: ", ma.max(skin_m_glob) 


#what is the min/max (for color table purposes it's good to know)
print "Deviation from averages by month and lat/lon...."
print "\nTOA min deviation: ", ma.min(toa_m_loc), "\nTOA max deviation: ", ma.max(toa_m_loc)
print "\nSkin temp min deviation: ", ma.min(skin_m_loc), "\nSkin temp max deviation: ", ma.max(skin_m_loc)

Deviation from global averages by month....

TOA min deviation:  -194.428390503 
TOA max deviation:  213.714996338

Skin temp min deviation:  -86.9217529297 
Skin temp max deviation:  39.5655517578
Deviation from averages by month and lat/lon....

TOA min deviation:  -189.832275391 
TOA max deviation:  123.84236145

Skin temp min deviation:  -24.3057556152 
Skin temp max deviation:  22.6296234131


### Deviation from year 2000: _base variables

This section calculates the deviations from the year 2000. The average for year 2000 is calculated at each latitude and longitude and used as the "baseline". This way we look at changes since the year 2000. This can also take into account months that we have data for but can't calculate the yearly average. 

TOA at position (1,1) in month 1/2000 - TOA at position (1,1) averaged over all months in the year 2000. 

In [385]:
# decided to add one more variable. This variable uses the first year (Jan 2000- Dec 2000) as a baseline. 
all_years = np.transpose(all_months)
skin_base = np.empty([182,180,360])
toa_base = np.empty([182,180,360])

for i in range(1,16):
    skin_base[all_years[i-1],:,:] = skin[all_years[i-1],:,:] - np.mean(skin[[0,1,2,3,4,5,6,7,8,9,10,11],:,:],0)
    toa_base[all_years[i-1],:,:] = toa[all_years[i-1],:,:] - np.mean(toa[[0,1,2,3,4,5,6,7,8,9,10,11],:,:],0)

skin_base[[180,181],:,:] = skin[[180,181],:,:] - np.mean(skin[[0,1,2,3,4,5,6,7,8,9,10,11],:,:],0)
toa_base[[180,181],:,:] = toa[[180,181],:,:] - np.mean(skin[[0,1,2,3,4,5,6,7,8,9,10,11],:,:],0)


#what is the min/max (for color table purposes it's good to know)
print "Deviation from averages by month and lat/lon...."
print "\nTOA min deviation: ", ma.min(toa_base), "\nTOA max deviation: ", ma.max(toa_base)
print "\nSkin temp min deviation: ", ma.min(skin_base), "\nSkin temp max deviation: ", ma.max(skin_base)

#for the line plots:
print "\nDeviation from temperature in 2000....\n"

for i in range(1,16):
    print "Year "+str(i-1)+": ", ma.average(skin[all_years[i-1],:,:]) - ma.average(skin[[all_years[0]],:,:])

Deviation from averages by month and lat/lon....

TOA min deviation:  -440.646484375 
TOA max deviation:  272.119720459

Skin temp min deviation:  -37.392868042 
Skin temp max deviation:  43.5535736084

Deviation from temperature in 2000....

Year 0:  0.0
Year 1:  0.133909465021
Year 2:  0.349773662551
Year 3:  0.198353909465
Year 4:  0.149691358025
Year 5:  0.454691358025
Year 6:  0.279670781893
Year 7:  0.280267489712
Year 8:  0.220185185185
Year 9:  0.218683127572
Year 10:  0.131255144033
Year 11:  0.219444444444
Year 12:  0.258230452675
Year 13:  0.318065843621
Year 14:  0.377757201646


### Deviations from the year 2000: _av variable

This section calculates the same baseline as the last section (average value at each latitude and longitude for the whole year 2000). In this instance however, we also calculate the same average for each year since then. 

TOA at position (1,1) for all months averaged in the year 2001 - TOA at position (1,1) for all months averaged in the year 2000.

In [386]:
# decided to add one more variable. This variable uses the first year (Jan 2000- Dec 2000) as a baseline. 
all_years = np.transpose(all_months)
skin_av = np.empty([182,180,360])
toa_av = np.empty([182,180,360])

for i in range(1,16):
    skin_av[all_years[i-1],:,:] = np.mean(skin[all_years[i-1],:,:],0) - np.mean(skin[[0,1,2,3,4,5,6,7,8,9,10,11],:,:],0)
    toa_av[all_years[i-1],:,:] = np.mean(toa[all_years[i-1],:,:],0) - np.mean(toa[[0,1,2,3,4,5,6,7,8,9,10,11],:,:],0)

#what is the min/max (for color table purposes it's good to know)
print "Deviation from averages by month and lat/lon...."
print "\nTOA min deviation: ", ma.min(toa_av), "\nTOA max deviation: ", ma.max(toa_av)
print "\nSkin temp min deviation: ", ma.min(skin_av), "\nSkin temp max deviation: ", ma.max(skin_av)

Deviation from averages by month and lat/lon....

TOA min deviation:  -29.673324585 
TOA max deviation:  44.7760314941

Skin temp min deviation:  -20.220489502 
Skin temp max deviation:  15.0378417969


## Saving the new data as a netCDF

This section takes our newly calculated variables and saves them into a netCDF format. If you calculate more variables, just add them using the format of a variable it is similar too. Just be sure to use a unique name. This section will not be necessary when we convert it to a visit script since we will just directly plot the data.

In [387]:
# Save new data as new netCDF file

#sources
#http://salishsea-meopar-tools.readthedocs.org/en/latest/netcdf4/
#http://nbviewer.jupyter.org/urls/bitbucket.org/salishsea/tools/raw/tip/I_ForcingFiles/Initial/PrepareTS.ipynb
#http://stackoverflow.com/questions/28462521/saving-climatology-result-to-netcdf-file


anom = Dataset('anomalies_4-14.nc','w')
anom.createDimension('time', 182)
anom.createDimension('lat', 180)
anom.createDimension('lon', 360)

anom_lat = anom.createVariable('lat', 'float32', ('lat'), zlib=True)
anom_lat.long_name='Latitude'
anom_lat.units = 'degrees_north'
anom_lat[:]= lat

anom_lon = anom.createVariable('lon', 'float32', ('lon'), zlib=True)
anom_lon.long_name='Longitude'
anom_lon.units = 'degrees_east'
anom_lon[:]= lon

anom_time = anom.createVariable('time', 'int32', ('time'), zlib=True)
anom_time.units = 'days since 2000-03-01 00:00:00'
anom_time.longname = 'time'
anom_time[:] = time

anom_toag = anom.createVariable('toa_glob', 'float32', ('time','lat','lon'), zlib=True, fill_value=-999.0)
anom_toag.units = 'W m-2'
anom_toag.longname = 'TOA Net flux all months, subtract global average'
anom_toag.coordinates = 'time lat lon'
anom_toag.valid_range= (-400,400)
anom_toag[:] = toa_glob

anom_toal = anom.createVariable('toa_loc', 'float32', ('time','lat','lon'), zlib=True, fill_value=-999.0)
anom_toal.units = 'W m-2'
anom_toal.longname = 'TOA Net flux all months, subtract lat/lon average'
anom_toal.coordinates = 'time lat lon'
anom_toal.valid_range= (-400,400)
anom_toal[:] = toa_loc

anom_skg = anom.createVariable('skin_glob', 'float32', ('time','lat','lon'), zlib=True, fill_value=-999.0)
anom_skg.units = 'K'
anom_skg.longname = 'Skin Temperature all months, subtract global average'
anom_skg.coordinates = 'time lat lon'
anom_skg.valid_range= (-400,400)
anom_skg[:] = skin_glob

anom_skl = anom.createVariable('skin_loc', 'float32', ('time','lat','lon'), zlib=True, fill_value=-999.0)
anom_skl.units = 'K'
anom_skl.longname = 'Skin Temperature all months, subtract lat/lon average'
anom_skl.coordinates = 'time lat lon'
anom_skl.valid_range= (-400,400)
anom_skl[:] = skin_loc

anom_toamg = anom.createVariable('toa_m_glob', 'float32', ('time','lat','lon'), zlib=True, fill_value=-999.0)
anom_toamg.units = 'W m-2'
anom_toamg.longname = 'TOA Net flux all months, subtract global average by month'
anom_toamg.coordinates = 'time lat lon'
anom_toamg.valid_range= (-400,400)
anom_toamg[:] = toa_m_glob

anom_toaml = anom.createVariable('toa_m_loc', 'float32', ('time','lat','lon'), zlib=True, fill_value=-999.0)
anom_toaml.units = 'W m-2'
anom_toaml.longname = 'TOA Net flux all months, subtract lat/lon average by month'
anom_toaml.coordinates = 'time lat lon'
anom_toaml.valid_range= (-400,400)
anom_toaml[:] = toa_m_loc

anom_skmg = anom.createVariable('skin_m_glob', 'float32', ('time','lat','lon'), zlib=True, fill_value=-999.0)
anom_skmg.units = 'K'
anom_skmg.longname = 'Skin Temperature all months, subtract global average by month'
anom_skmg.coordinates = 'time lat lon'
anom_skmg.valid_range= (-400,400)
anom_skmg[:] = skin_m_glob

anom_skml = anom.createVariable('skin_m_loc', 'float32', ('time','lat','lon'), zlib=True, fill_value=-999.0)
anom_skml.units = 'K'
anom_skml.longname = 'Skin Temperature all months, subtract lat/lon average by month'
anom_skml.coordinates = 'time lat lon'
anom_skml.valid_range= (-400,400)
anom_skml[:] = skin_m_loc

anom_toamb = anom.createVariable('toa_base', 'float32', ('time','lat','lon'), zlib=True, fill_value=-999.0)
anom_toamb.units = 'W m-2'
anom_toamb.longname = 'TOA Net flux all months, subtract year 2000'
anom_toamb.coordinates = 'time lat lon'
anom_toamb.valid_range= (-400,400)
anom_toamb[:] = toa_base

anom_skmb = anom.createVariable('skin_base', 'float32', ('time','lat','lon'), zlib=True, fill_value=-999.0)
anom_skmb.units = 'K'
anom_skmb.longname = 'Skin Temperature all months, subtract year 2000'
anom_skmb.coordinates = 'time lat lon'
anom_skmb.valid_range= (-400,400)
anom_skmb[:] = skin_base

anom_toamb = anom.createVariable('toa_av', 'float32', ('time','lat','lon'), zlib=True, fill_value=-999.0)
anom_toamb.units = 'W m-2'
anom_toamb.longname = 'TOA Net flux all months, subtract year 2000'
anom_toamb.coordinates = 'time lat lon'
anom_toamb.valid_range= (-400,400)
anom_toamb[:] = toa_av

anom_skmb = anom.createVariable('skin_av', 'float32', ('time','lat','lon'), zlib=True, fill_value=-999.0)
anom_skmb.units = 'K'
anom_skmb.longname = 'Skin Temperature all months, subtract year 2000'
anom_skmb.coordinates = 'time lat lon'
anom_skmb.valid_range= (-400,400)
anom_skmb[:] = skin_av

anom.close()

## Check the output

The output opens with Visit (at least for me it works fine). This section is to also check that the netCDF file actually stored our new data. 

In [308]:
# Open new netCDF file to check that it stored things
ds2 = Dataset('anomalies.nc','r')
print(ds2)
print(ds2.dimensions)
print(ds2.variables['toa_glob'][1,:,:])

<type 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): time(182), lat(180), lon(360)
    variables(dimensions): float32 lat(lat), float32 lon(lon), int32 time(time), float32 toa_glob(time,lat,lon), float32 toa_loc(time,lat,lon), float32 skin_glob(time,lat,lon), float32 skin_loc(time,lat,lon), float32 toa_m_glob(time,lat,lon), float32 toa_m_loc(time,lat,lon), float32 skin_m_glob(time,lat,lon), float32 skin_m_loc(time,lat,lon)
    groups: 

OrderedDict([(u'time', <type 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 182
), (u'lat', <type 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 180
), (u'lon', <type 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 360
)])
[[ 6.04196548  6.04196548  6.04196548 ...,  6.04196548  6.04196548
   6.04196548]
 [ 6.08597803  6.08597803  6.08597803 ...,  5.62767887  5.62767887
   5.62767887]
 [ 3.86646509  3.86646509  3.86646509 ...,  3.39739656  3.39739656
   3.39739656]
 ..., 
 [-3.05375814